In [ ]:
import pandas as pd
import re
from datasets import Dataset

In [2]:
df = pd.read_csv('../data/naruto.csv')
df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [ ]:
def clean_text(text):
    return re.sub(r'\(.*?\)', '', text)
df['line'] = df['line'].apply(clean_text)
df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [5]:
def count_words(text):
    return len(text.strip().split(" "))
df['word_count'] = df['line'].apply(count_words)
df.head()

,name,line,word_count
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [13]:
df.drop(columns=['seleted'], inplace=True)
df['selected'] = 0
df.loc[(df['name'] == "Naruto") & (df['word_count'] > 5), 'selected'] = 1
df.head()

,name,line,word_count,selected
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0


In [16]:
selected_indx = df[(df['selected'] == 1) & (df.index>0)].index.tolist()
selected_indx[:5]

[6, 28, 30, 32, 36]

In [18]:
system_prompt = "You are a Naruto from the anime Naruto. Answer the questions in the style of Naruto Uzumaki."
prompts = []
for i in selected_indx:
    prompt = system_prompt + "\n"
    prompt += df.iloc[i-1]['line'] + "\n"
    prompt += df.iloc[i]['line'] 
    prompts.append(prompt)

prompts[:4]

['You are a Naruto from the anime Naruto. Answer the questions in the style of Naruto Uzumaki.\n  Oh yeah, Naruto!?\n Where’d you come from, Iruka Sensei!? What are you doing here?',
 'You are a Naruto from the anime Naruto. Answer the questions in the style of Naruto Uzumaki.\n You’re not going home till you’ve cleaned off every single drop of paint.\n  So what? It’s not like there’s anyone waiting at home for me. ',
 'You are a Naruto from the anime Naruto. Answer the questions in the style of Naruto Uzumaki.\n  Naruto?\n  What do you want now, Sensei?',
 'You are a Naruto from the anime Naruto. Answer the questions in the style of Naruto Uzumaki.\n Well, I was just thinking. Maybe after you clean this all up I can take you out for some ramen. The good stuff. What do you think? \n  Now that’s some serious motivation! I’ll have this clean in no time!']

In [19]:
df_prompt = pd.DataFrame({'prompt': prompts})
df_prompt.head()

,prompt
0,You are a Naruto from the anime Naruto. Answer...
1,You are a Naruto from the anime Naruto. Answer...
2,You are a Naruto from the anime Naruto. Answer...
3,You are a Naruto from the anime Naruto. Answer...
4,You are a Naruto from the anime Naruto. Answer...


In [ ]:
dataset = Dataset.from_pandas(df_prompt)